In [104]:
import pandas as pd
from datetime import datetime as dt

In [3]:
df=pd.read_csv('C:/Users/hp/OneDrive/Desktop/datasets/netflix_titles.csv')

In [ ]:
df.head(5)

In [6]:
df.shape

(8807, 12)

In [ ]:
df.info()


In [10]:
df.size

105684

In [19]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added         0
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [ ]:
#converting into datetime format
df['date_added']=pd.to_datetime(df['date_added'],format='mixed')

In [26]:
df.dtypes

show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             int64
rating                  object
duration                object
listed_in               object
description             object
dtype: object

In [28]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added         0
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

since the duration is in the form of "90 min" its not a good practice for analysis
And not delteing the duration column for future debugging

In [29]:
df[['duration_value', 'duration_unit']]=df['duration'].str.extract(r'(\d+)\s*(\w+)')

In [ ]:
print(df[['duration_value','duration_unit']])

removing the duplicated from the dataset

In [ ]:

print("before the removal of duplicates:",df.duplicated().sum())

before the removal of duplicates: 0


In [36]:
df=df.drop_duplicates()

In [115]:
print("after the removal of duplicates:",df.duplicated().sum())

after the removal of duplicates: 0


In [38]:
df.drop(columns=['description'],inplace=True)

In [71]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,duration_value,duration_unit
0,s1,Movie,Dick Johnson Is Dead,Not Specified,Not Specified,None,2021-09-25,2020,PG-13,90 min,Documentaries,90,min
1,s2,TV Show,Blood & Water,Not Specified,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",None,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",2,Seasons
2,s3,TV Show,Ganglands,Not Specified,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",1,Season
3,s4,TV Show,Jailbirds New Orleans,Not Specified,Not Specified,None,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV",1,Season
4,s5,TV Show,Kota Factory,Not Specified,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",None,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",2,Seasons


Standardizing the values of director based on the pairs of director and cast in the dataset
And only the values that are repeated thrice

In [44]:
# Drop rows where either column is missing to avoid NaN issues
valid_df = df.dropna(subset=['director', 'cast'])

# Get frequent (cast -> director) mappings
cast_director_counts = valid_df.groupby(['cast', 'director']).size()
cast_director_counts = cast_director_counts[cast_director_counts >= 3]

# Get the most frequent director for each cast
cast_to_director = cast_director_counts.groupby(level=0).idxmax().apply(lambda x: x[1])

# Fill in missing directors using this mapping
df['director'] = df.apply(
    lambda row: cast_to_director[row['cast']] if pd.isna(row['director']) and row['cast'] in cast_to_director else row['director'],
    axis=1
)


In [43]:
df.isnull().sum()

show_id             0
type                0
title               0
director            0
cast              825
country           831
date_added          0
release_year        0
rating              4
duration            3
listed_in           0
duration_value      3
duration_unit       3
dtype: int64

Here filling the remaining the Nan values are filled wiht "not specified"

In [56]:
#filling the missing values in the 'director' column with 'not specified'
df['director'] = df['director'].fillna('Not Specified')


In [47]:
directors = df['director']
countries = df['country']

In [49]:
pairs = zip(directors, countries)
dir_cntry = dict(list(pairs))

In [ ]:
for director, country in dir_cntry.items():
    df.loc[(df['country'].isna()) & (df['director'] == director), 'country'] = country


In [ ]:
df.isnull().sum()

In [74]:
#filling the missing values in the 'country' column
df['country'] = df['country'].fillna('Not Specified')

In [77]:
df['cast'] = df['cast'].fillna('Not Specified')

In [85]:
df['director']=df['director'].fillna('Not specifited')

Dropping the columns that are Nan values using the bool method 

In [87]:
df = df[df['date_added'].notna()]


In [ ]:
df['date_added'].isna().sum()

In [89]:
df=df[df['rating'].notna()]

In [ ]:
df['rating'].isna().sum()

In [92]:
df=df[df['duration'].notna()]

finally checking any null values in the dataset

In [102]:
df.isna().sum()

show_id           0
type              0
title             0
director          0
cast              0
country           0
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
duration_value    0
duration_unit     0
dtype: int64

In [101]:
df['country']=df['country'].fillna('Not specified')

The below cells are to find out the incosistencies 
Here there are relased years that are preceeded that the date_added
 

In [105]:
sum(df['date_added'].dt.year<df['release_year'])

14

In [112]:
df.loc[df['date_added'].dt.year < df['release_year'], ['date_added', 'release_year']]

,date_added,release_year
1551,2020-12-14,2021
1696,2020-11-15,2021
2920,2020-02-13,2021
3168,2019-12-06,2020
3287,2019-11-13,2020
3369,2019-10-25,2020
3433,2019-10-11,2020
4844,2018-05-30,2019
4845,2018-05-29,2019
5394,2017-07-01,2018


In [120]:
import os

output_path = 'c:/users/hp/onedrive/desktop/datasets/cleaned_netflix.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)
print(f"\n=== CLEANED DATASET SAVED TO: {output_path} ===")


=== CLEANED DATASET SAVED TO: c:/users/hp/onedrive/desktop/datasets/cleaned_netflix.csv ===
